In [30]:
import json
from datasets import Dataset, load_dataset, concatenate_datasets


In [31]:
tags_dict = json.load(open("tags_dict.json"))

In [32]:
tags_dict["jutsu"]

['Adamantine Sealing Chains',
 'All Fire Style Jutsu in Naruto',
 'Body Coating Technique',
 'Boruto Jutsu',
 'Earth Style Jutsu',
 'Fire Style Jutsu',
 'Fire Style: Fireball Jutsu in Naruto',
 'Flying Thunder God Technique in Naruto',
 'Forbidden Jutsu in Naruto',
 'Four Red Yang Formation',
 'Fuinjutsu',
 "Haku's One Hand Jutsu",
 'Hand Signs in Naruto',
 'Harem Jutsu',
 'Hidden Mist Jutsu',
 'Kamui',
 'Lightning Jutsu in Naruto',
 'Lightning Style jutsu',
 'Lightning Style Rasengan in Naruto',
 'Minato Namikaze Jutsus',
 'Mind Transfer Jutsu in Naruto',
 'Mini-Rasenshuriken',
 'Mystical Palm Jutsu in Naruto',
 'Naruto Akatsuki Members',
 'Naruto’s Summoning Jutsu',
 'Oiroke no Jutsu',
 'Orochimaru’s Jutsu',
 'Rasenshuriken',
 'Reanimation Jutsu in Naruto',
 'Reaper Death Seal Jutsu',
 'Rock Lee’s Jutsu',
 'Sakura’s Jutsu',
 'Shadow Clone Jutsu',
 'Substitution Jutsu',
 'Summoning Jutsu',
 'Tobirama Senju’s Jutsu',
 'Water Dragon Jutsu in Naruto',
 'What are all the Sharingans in Nar

In [33]:
keywords = set()
for tag in tags_dict:
    for keyword in tags_dict[tag]:
        keywords.add(keyword.lower())

In [34]:
keywords = set(["naruto", "jutsu", "raikage", "sasuke", "uchiha", "kakashi", "hokage", "konoha", "sharingan", "jinchuriki", "akatsuki", "uzumaki", "sakura", "sasori", "itachi", "orochimaru", "jiraiya", "kage", "shippuden", "kisame", "deidara", "gaara", "neji", "rock lee", "tenten", "shikamaru", "ino", "choji", "kiba", "hinata", "shino", "sai", "yamato", "anbu", "tsunade", "inoichi", "shizune", "kurenai", "asuma", "konohamaru", "kabuto", "temari", "kankuro", "karin", "suigetsu", "juugo", "sasuke", "itachi", "madara", "hashirama", "tobirama", "hiruzen", "minato", "kushina", "obito", "rin", "kakashi", "gai", "asuma", "kurenai", "shizune", "inoichi", "shikaku", "choza", "hiashi", "hizashi", "kiba", "shino", "hinata", "neji", "tenten", "rock lee", "sakura", "ino", "shikamaru", "choji", "naruto", "konohamaru", "jiraiya", "tsunade", "orochimaru", "kabuto", "sasori", "deidara", "kisame", "itachi", "kakuzu", "hidan", "konan", "pein", "zetsu", "tobi", "madara", "hashirama", "tobirama", "hiruzen", "minato", "kushina", "obito", "rin", "kakashi", "gai", "asuma", "kurenai", "shizune", "inoichi", "shikaku", "choza", "hiashi", "hizashi", "kiba", "shino", "hinata"])

In [35]:
quora_dataset = load_dataset("toughdata/quora-question-answer-dataset")

In [38]:
quora_dataset["train"]

Dataset({
    features: ['question', 'answer'],
    num_rows: 56402
})

In [7]:
instruction = "Answer the question"


In [9]:
naruto_ds = quora_dataset["train"].filter(lambda d: any([keyword in d["question"].lower().split() for keyword in keywords]), batched=False)
print(len(naruto_ds))
# preview top n
n = 20
for i, example in enumerate(naruto_ds):
    print(f"Question: {example['question']}")
    print(f"Answer: {example['answer']}")
    for keyword in keywords:
        if keyword in example['question'].lower():
            print(f"Keyword: {keyword}")
    print("---")
    if i == n:
        break

Filter: 100%|██████████| 56402/56402 [00:02<00:00, 25507.95 examples/s]

52
Question: What are some budget hotels near the Sirdi Sai Baba Temple?
Answer: There are lot of hotels around Shirdi Sai Baba Temple ,Kindly search online, you will get plenty of hotels of any budget, and apart from that Sai Sanasthan has also big accomodation facility of all budget, you can book through website—saibaba.org.
 Must go to Shirdi ,spent few days there, participate in Aarti, Palki on Thursday, Shri Satyanaryan pooja, have prasad at Prasadalya .You will feel much blessed.
 OM SAIRAM

Keyword: sai
---
Question: If obito didnt get killed by kaguya, would he have still survived since he performed rinne-rebirth?
Answer: I do not think so.
 The first time we saw this ability in use, Nagato used all of his remaining strength after the Destruction of Konoha and his fight with Naruto to summon the King of Hell and restore life to those who had died during the attack.
Nagato died after performing this technique, with a thought by Konan that seemed to indicate that Nagato would not

In [15]:
prompts = []
for row in naruto_ds:
    prompts.append({"instruction": instruction, "input": row["question"], "response": row["answer"]})

In [19]:
with open("prompt_quora_v1.jsonl", "w") as f:
    for prompt in prompts:
        f.write(json.dumps(prompt) + "\n")

In [39]:
msmarco_ds = load_dataset("ms_marco", "v2.1")

In [40]:
naruto_msmacro_ds = concatenate_datasets([msmarco_ds[v].filter(lambda d: any([keyword in d["query"].lower().split() for keyword in keywords]), batched=False, num_proc=4) for v in ["train", "validation", "test"]])
print(len(naruto_msmacro_ds))
# preview top n
n = 20
for i, example in enumerate(naruto_msmacro_ds):
    print(f"Question: {example['query']}")
    content = example['passages']["passage_text"]
    print(f"Answer: {content}")
    for keyword in keywords:
        if keyword in example['query'].lower().split():
            print(f"Keyword: {keyword}")
    print("---")
    if i == n:
        break

171
Question: how many seasons are in the first naruto
Answer: ['The number of episodes of Naruto in each season varied but there were 220 episodes made, split over 5 seasons. In addition, there have been 3 movies released, all of which a…re available on DVD.', 'Season 8 - 25 episodes Season 9 - 11 episodes ---- That is it for now. Visit: http://en.wikipedia.org/wiki/Naruto_episode_guide for more. How many episodes are there in the first season of Dexter?', "Created by Masashi Kishimoto. With Junko Takeuchi, Maile Flanagan, Kate Higgins, Chie Nakamura. Naruto Uzumaki, an adolescent ninja, struggles as he searches for recognition and dreams of becoming the Hokage, the village's leader and strongest ninja. Creator: Masashi Kishimoto", "Title: Naruto (2002–2007) 8.2 /10. Want to share IMDb's rating on your own site? Use the HTML below.", 'Bonanza was a drama/western series that aired for a total of 14 years (1959 - 1973).The first season was 32 episodes long, which seems very long compare

In [46]:
naruto_msmacro_ds[5]

{'answers': ['Kaguya clan'],
 'passages': {'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
  'passage_text': ['Fūma Clan (Land of Sound) Main article: Fūma Clan (Land of Sound) The Fūma Clan (ふうま一族, Fūma Ichizoku) is a shinobi clan from the Land of the Rice Fields, where Orochimaru established the Land of Sound. The clan specialises in using chakra threads and also have a large number of shinobi trained in archery.',
   "Sasuke is one of the several surviving members of the Uchiha clan, an infamous and powerful extended family of ninjas who were allied to the village of Konohagakure, before being massacred by Sasuke's older brother, Itachi Uchiha, before the series' start.",
   "During the Second Shinobi World War, Orochimaru joined Jiraiya and Tsunade in fighting Hanzō, who gave them the title of Konoha's Three Legendary Shinobi (伝説の三忍, Densetsu no Sannin) as a reward for surviving their battle with him, while the rest of their group were easily slaughtered in the process.",
   "Orochi

In [28]:
prompts = []
for row in naruto_msmacro_ds:
    prompts.append({"instruction": instruction, "input": row["query"] + "?", "response": row["passages"]["passage_text"]})

In [29]:
with open("prompt_msmarco_v1.jsonl", "w") as f:
    for prompt in prompts:
        f.write(json.dumps(prompt) + "\n")